# First Class Function in Python
[Reference](https://realpython.com/primer-on-python-decorators)
## What is a first class function?
A programming language is said to have first-class functions if it treats functions as first-class citizens. In Python, functions are first-class objects. First-class functions are a powerful programming concept that allows functions to be passed as arguments to other functions, returned as values from other functions, and assigned to variables.

In [6]:
def yell(text):
    return text.upper() + '!'

In [7]:
bark = yell
funcs = [bark, str.lower, str.capitalize]
funcs

[<function __main__.yell(text)>,
 <method 'lower' of 'str' objects>,
 <method 'capitalize' of 'str' objects>]

In [8]:
for f in funcs:
    print(f, f('hey there'))

<function yell at 0x7ff8c422dcf0> HEY THERE!
<method 'lower' of 'str' objects> hey there
<method 'capitalize' of 'str' objects> Hey there


In [9]:
def greet(func):
    greeting = func('Hi, I am a Python program')
    print(greeting)

In [10]:
greet(bark)

HI, I AM A PYTHON PROGRAM!


In [11]:
list(map(yell, ['hello', 'hey', 'hi']))

['HELLO!', 'HEY!', 'HI!']

In [12]:
def get_speak_func(volume):
    def whisper(text):
        return text.lower() + '...'
    def yell(text):
        return text.upper() + '!'
    if volume > 0.5:
        return yell
    else:
        return whisper
    
get_speak_func(0.6)('hello')

'HELLO!'

This is a lexical closure problem. We need to implement a function that will take a string and volume of the speaker and return a new string with the volume of the speaker. The volume of the speaker is the number of times the string is repeated.

In [13]:
def get_speak_func(text, volume):
    def whisper():
        return [text.lower() + '...' for i in range(int(volume*10))]
    def yell():
        return [text.upper() + '!' for i in range(int(volume*10))]
    if volume > 0.5:
        return yell
    else:
        return whisper
    
get_speak_func('Hello, World', 0.7)()

['HELLO, WORLD!',
 'HELLO, WORLD!',
 'HELLO, WORLD!',
 'HELLO, WORLD!',
 'HELLO, WORLD!',
 'HELLO, WORLD!',
 'HELLO, WORLD!']

In [14]:
def make_adder(n):
    def add(x):
        return x + n
    return add

plus_3 = make_adder(3)
plus_5 = make_adder(5)

plus_3(4), plus_5(4) 

(7, 9)

Here is a simplified example of how a first-class function in Python can capture local state

In [15]:
n = 10
def add(x):
    return x + n

print(add(10))

20


Objects Can Behave Like Functions 
Behind the scenes, "calling" an object instance invokes the `__call__` method on the object's class. This is a special method that can be defined in a class to allow the class's instances.

But not all objects can be called,, you can check with the `callable()` function.

In [16]:
class Adder:
    def __init__(self, n):
        self.n = n

    def __call__(self, x):
        return self.n + x
    
plus_3 = Adder(3)
print(plus_3)

plus_3(4)

7

In [17]:
class Adder_not_callable:
    def __init__(self, n):
        self.n = n
add = Adder_not_callable(3)

In [18]:
print(f"Callable {callable(plus_3)}")
print(f"Callable {callable(add)}")

Callable True
Callable False


Key Takeaways
Everything in Python is an object, including functions. You can assign them to variables, store them in data structures, and pass or return them to and from other functions (first-class functions.)

First-class functions allow you to abstract away and pass around behavior in your programs.

Functions can be nested and they can capture and carry some of the parent function’s state with them. Functions that do this are called closures.
Objects can be made callable which allows you to treat them like functions in many cases.

# Decorator

A decorator is a design pattern in Python that allows a user to add new functionality to an object without modifying its structure. Decorators are usually called before the definition of a function you want to decorate.

## Concepts
- Higher Order Function: A function that takes a function as an argument or returns a function.

In [19]:
def decorator(func: object) -> object:
    def wrapper() -> None:
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = decorator(say_whee)
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


In [20]:
def decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@decorator
def say_whee():
    print("Whee!")
    print("Whee!")

say_whee()


Something is happening before the function is called.
Whee!
Whee!
Something is happening after the function is called.


In [21]:
import functools

def do_twice(func):
    @functools.wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def say_whee():
    print("Whee!")

@do_twice
def greet(name):
    print(f"Hello {name}")

@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

test_list =  []
test_list.append(return_greeting("Adam"))
test_list

Creating greeting
Creating greeting


['Hi Adam']

In [22]:
help(say_whee)

Help on function say_whee in module __main__:

say_whee()



# Real World Example

Good boilerplate for decorators:
```python
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

In [23]:
import functools
import time

class Logger:
    _instance = None
    _func = None

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super().__new__(cls)
        return cls._instance

def timer(func):
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Finished {func.__name__} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10000)])

if "__main__" == __name__:
    waste_some_time(1)
    waste_some_time(999)

Finished waste_some_time in 0.0159 secs


Finished waste_some_time in 5.3070 secs


In [24]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f"{k}={repr(v)}" for k, v in kwargs.items()]
        signature = ", ".join(args_repr + kwargs_repr)
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__}() returned {repr(value)}")
        return value
    return wrapper_debug


@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already, you are growing up!"
    
if "__main__" == __name__:
    make_greeting("Benjamin")
    make_greeting("Richard", age=112)

Calling make_greeting('Benjamin')
make_greeting() returned 'Howdy Benjamin!'
Calling make_greeting('Richard', age=112)
make_greeting() returned 'Whoa Richard! 112 already, you are growing up!'


In [25]:
import math

math.factorial = debug(math.factorial) #manually decorate the math.factorial function

def approximate_e(terms=18):
    return sum((1 / math.factorial(n)) for n in range(terms))

if "__main__" == __name__:
    approximate_e(5)

Calling factorial(0)
factorial() returned 1
Calling factorial(1)
factorial() returned 1
Calling factorial(2)
factorial() returned 2
Calling factorial(3)
factorial() returned 6
Calling factorial(4)
factorial() returned 24


In [26]:
import functools
import time

def slow_down(func):
    """Sleep 1 second before calling the function"""
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

@slow_down
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

if "__main__" == __name__:
    countdown(3)

3
2
1
Liftoff!


In [27]:
PLUGINS = dict()

def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return f"Hello {name}"


@register
def be_awesome(name):
    return f"Yo {name}, together we're the awesomest!"

import random  

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}") #same as repr(greeter)
    return greeter_func(name)

if "__main__" == __name__:
    randomly_greet("Alice")

Using 'be_awesome'


In [28]:
globals()

{'__name__': '__main__',
 '__doc__': 'Automatically created module for IPython interactive environment',
 '__package__': None,
 '__loader__': None,
 '__spec__': None,
 '__builtin__': <module 'builtins' (built-in)>,
 '__builtins__': <module 'builtins' (built-in)>,
 '_ih': ['',
  'PLUGINS = dict()\n\ndef register(func):\n    """Register a function as a plug-in"""\n    PLUGINS[func.__name__] = func\n    return func\n\n@register\ndef say_hello(name):\n    return f"Hello {name}"\n\n\n@register\ndef be_awesome(name):\n    return f"Yo {name}, together we\'re the awesomest!"\n\nimport random  \n\ndef randomly_greet(name):\n    greeter, greeter_func = random.choice(list(PLUGINS.items()))\n    print(f"Using {greeter!r}") #same as repr(greeter)\n    return greeter_func(name)\n\nif "__main__" == __name__:\n    randomly_greet("Alice")',
  'PLUGINS = dict()\n\ndef register(func):\n    """Register a function as a plug-in"""\n    PLUGINS[func.__name__] = func\n    return func\n\n@register\ndef say_hel

# Review code

Pie notation is a shorthand for calling a function. It is a way to call a function without using parentheses. It is used to call a function that takes no arguments.

repr() is a built-in function in Python that returns a string representation of an object. The repr() function returns a printable representation of the given object.

globals and decorator are used to get the global variables and the decorator function.


## Fancy Decorators

- [ ] Need to look more into python build-in decorators
- [ ] Need to review this code

In [29]:
# Here is a example of buildin decorators in python
class Circle:
    def __init__(self, radius):
        self.radius = radius

    @property
    def radius(self):
        """Get value of radius"""
        return self._radius

    @radius.setter
    def radius(self, value):
        """Set radius, raise error if negative"""
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("radius must be non-negative")

    @property
    def area(self):
        """Calculate area inside circle"""
        return self.pi() * self.radius**2

    def cylinder_volume(self, height):
        """Calculate volume of cylinder with circle as base"""
        return self.area * height

    @classmethod
    def unit_circle(cls):
        """Factory method creating a circle with radius 1"""
        return cls(1)

    @staticmethod
    def pi():
        """Value of π, could use math.pi instead though"""
        return 3.1415926535

In [30]:
c = Circle(5)
c.radius =  10


In [31]:
class TimeWaster:
    @debug
    def __init__(self, max_num):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([number**2 for number in range(self.max_num)])


tw = TimeWaster(1000)
tw.waste_time(999)

Calling __init__(<__main__.TimeWaster object at 0x7ff8c431e950>, 1000)
__init__() returned None
Finished waste_time in 0.6036 secs


In [32]:
# This way I only meansure the for class instance creation and method call
@timer
class TimeWaster:
    def __init__(self, max_num):
        self.max_num = max_num

    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

In [6]:
def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f"Call {wrapper_count_calls.num_calls} of {func.__name__}()")
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls

@count_calls
def say_whee():
    print("Whee!")


say_whee()
say_whee()
say_whee.num_calls


Call 1 of say_whee()
Whee!
Call 2 of say_whee()
Whee!


2

In [4]:
import functools

class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__}()")
        return self.func(*args, **kwargs)

@CountCalls
def say_whee():
    print("Whee!")

say_whee()
say_whee()

Call 1 of say_whee()
Whee!
Call 2 of say_whee()
Whee!


In [38]:
import functools
import time

# ...

def slow_down(_func=None, *, rate=1):
    """Sleep given amount of seconds before calling the function"""
    def decorator_slow_down(func):
        @functools.wraps(func)
        def wrapper_slow_down(*args, **kwargs):
            time.sleep(rate)
            return func(*args, **kwargs)
        return wrapper_slow_down

    if _func is None:
        return decorator_slow_down
    else:
        return decorator_slow_down(_func)

@slow_down(rate=2)
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

countdown(3)

3
2
1
Liftoff!


In [39]:
def singleton(cls):
    """Make a class a Singleton class (only one instance)"""
    @functools.wraps(cls)
    def wrapper_singleton(*args, **kwargs):
        if wrapper_singleton.instance is None:
            wrapper_singleton.instance = cls(*args, **kwargs)
        return wrapper_singleton.instance
    wrapper_singleton.instance = None
    return wrapper_singleton


@singleton
class TheOne:
    pass


first_one = TheOne()
another_one = TheOne()

id(first_one)


id(another_one)


first_one is another_one

True

In [15]:
def cache(func):
    """Keep a cache of previous function calls"""
    @functools.wraps(func)
    def wrapper_cache(*args, **kwargs):
        cache_key = args + tuple(kwargs.items())
        if cache_key not in wrapper_cache.cache:
            wrapper_cache.cache[cache_key] = func(*args, **kwargs)
        return wrapper_cache.cache[cache_key]
    wrapper_cache.cache = {}
    return wrapper_cache

@cache
@count_calls
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

fibonacci(10)

Call 1 of fibonacci()
Call 2 of fibonacci()
Call 3 of fibonacci()
Call 4 of fibonacci()
Call 5 of fibonacci()
Call 6 of fibonacci()
Call 7 of fibonacci()
Call 8 of fibonacci()
Call 9 of fibonacci()
Call 10 of fibonacci()
Call 11 of fibonacci()


55

In [19]:
fibonacci(3)

CacheInfo(hits=11, misses=15, maxsize=4, currsize=4)

Im using a cache for exemple of decorators, but dont write my on code when need to use it, just use build-in decorators.

Here [lru cache python](https://realpython.com/lru-cache-python/)


In [20]:
@functools.lru_cache(maxsize=4)
def fibonacci(num):
    if num < 2:
        value = num
    else:
        value = fibonacci(num - 1) + fibonacci(num - 2)
    print(f"Calculated fibonacci({num}) = {value}")
    return value

fibonacci(10)
print("________________________")
fibonacci(8)
fibonacci.cache_info()

Calculated fibonacci(1) = 1
Calculated fibonacci(0) = 0
Calculated fibonacci(2) = 1
Calculated fibonacci(3) = 2
Calculated fibonacci(4) = 3
Calculated fibonacci(5) = 5
Calculated fibonacci(6) = 8
Calculated fibonacci(7) = 13
Calculated fibonacci(8) = 21
Calculated fibonacci(9) = 34
Calculated fibonacci(10) = 55
________________________


CacheInfo(hits=9, misses=11, maxsize=4, currsize=4)

In [24]:
import math
import pint

def set_unit(unit):
    """Register a unit on a function"""
    def decorator_set_unit(func):
        func.unit = unit
        return func
    return decorator_set_unit

@set_unit("cm^3")
def volume(radius, height):
    return math.pi * radius**2 * height

ureg = pint.UnitRegistry()
vol = volume(3, 5) * ureg("cm^3")

vol.to("cubic inches")

vol.to("gallons").m


0.0373464440537444